In [1]:
!pip install keras-drop-block

  Preparing metadata (setup.py) ... done
  Created wheel for keras-drop-block: filename=keras_drop_block-0.7.0-py3-none-any.whl size=6253 sha256=55f6cad35d7e9462a718cc3f7780a2f88327d366f2e1205e0af0f70a3b52da9e
  Stored in directory: /Users/rakibuddin/Library/Caches/pip/wheels/91/2a/81/77fa625804f1ff5bbe6b6f088e8153f9c4c9a379bf5061f151
Successfully built keras-drop-block


In [2]:
from numpy.random import seed
seed(1)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import itertools
import tensorflow
import tensorflow.keras
from tensorflow.keras.applications import ResNet152, Xception,VGG16,EfficientNetB4
from tensorflow.keras.models import Sequential, Model
#from tensorflow.keras.layers import Activation,Dense, Dropout, Flatten, Conv2D, MaxPool2D,AveragePooling2D,GlobalMaxPooling2D
from tensorflow.keras.layers import Activation,Dense, Dropout, Flatten, Conv2D, MaxPool2D,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
np.random.seed(123)
print("done")

ImportError: cannot import name 'EfficientNetB4' from 'tensorflow.keras.applications' (/Users/rakibuddin/opt/anaconda3/envs/Deep/lib/python3.7/site-packages/tensorflow_core/python/keras/api/_v2/keras/applications/__init__.py)

In [3]:
lesion_type_dict = {'akiec': 'Actinic keratoses',
                    'bcc': 'Basal cell carcinoma',
                    'bkl': 'Benign keratosis-like lesions ',
                    'df': 'Dermatofibroma',
                    'nv': 'Melanocytic nevi',
                    'mel': 'Melanoma',
                    'vasc': 'Vascular lesions'}

base_skin_dir = os.path.join('..', 'input')
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in glob(os.path.join(base_skin_dir, '*','*', '*.jpg'))}
skin_df = pd.read_csv('../input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')
skin_df['path'] = skin_df['image_id'].map(imageid_path_dict.get)
skin_df['cell_type'] = skin_df['dx'].map(lesion_type_dict.get) 
skin_df['cell_type_idx'] = pd.Categorical(skin_df['cell_type']).codes
skin_df.groupby(['dx']).count()

,lesion_id,image_id,dx_type,age,sex,localization,path,cell_type,cell_type_idx
dx,,,,,,,,,
akiec,327,327,327,327,327,327,327,327,327
bcc,514,514,514,514,514,514,514,514,514
bkl,1099,1099,1099,1089,1099,1099,1099,1099,1099
df,115,115,115,115,115,115,115,115,115
mel,1113,1113,1113,1111,1113,1113,1113,1113,1113
nv,6705,6705,6705,6660,6705,6705,6705,6705,6705
vasc,142,142,142,142,142,142,142,142,142


In [ ]:
skin_df['image'] = skin_df['path'].map(lambda x: np.asarray(Image.open(x).resize((75,100))))
features=skin_df.drop(columns=['cell_type_idx'],axis=1)
target=skin_df['cell_type_idx']

In [ ]:
from tensorflow.keras.utils import to_categorical
x_train_o, x_test_o, y_train_o, y_test_o = train_test_split(features, target, test_size=0.2,random_state=666)
x_train = np.asarray(x_train_o['image'].tolist())
x_test = np.asarray(x_test_o['image'].tolist())
x_train_mean = np.mean(x_train)
x_train_std = np.std(x_train)
x_test_mean = np.mean(x_test)
x_test_std = np.std(x_test)
x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_test_mean)/x_test_std
y_train = to_categorical(y_train_o, num_classes = 7)
y_test = to_categorical(y_test_o, num_classes = 7)
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.1, random_state = 999)

In [ ]:
#Reshaping the Images into 3 channels (RGB)
x_train = x_train.reshape(x_train.shape[0], *(75, 100, 3))
x_test = x_test.reshape(x_test.shape[0], *(75, 100, 3))
x_validate = x_validate.reshape(x_validate.shape[0], *(75, 100, 3))

In [ ]:
input_shape = (75, 100, 3)
num_classes = 7
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=4, verbose=1, factor=0.5, min_learning_rate=0.00001)

#Data Generation
datagen = ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False, 
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=90,
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        shear_range = 10) 
datagen.fit(x_train)

# ***VGG16 HeadModel_DO***

In [ ]:
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.layers import BatchNormalization
from keras_drop_block import DropBlock2D
baseModel = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
headModel = baseModel.output

headModel_DO = Dropout(0.5)(headModel)
headModel_DO = BatchNormalization()(headModel_DO)
headModel_DO = AveragePooling2D()(headModel_DO)
headModel_DO = Flatten(name="flatten")(headModel_DO)
headModel_DO = Dense(64, activation="relu")(headModel_DO)
headModel_DO = Dropout(0.5)(headModel_DO)
headModel_DO = Dense(7, activation="softmax")(headModel_DO)
model_DO = Model(inputs=baseModel.input, outputs=headModel_DO)

for layer in baseModel.layers:
    layer.trainable = False
    
model_DO.summary()

In [ ]:
model_DO.compile(loss="categorical_crossentropy", optimizer=optimizer,
    metrics=["accuracy"])

H_DO = model_DO.fit(datagen.flow(x_train,y_train, batch_size=20),
                             epochs = 50, validation_data = (x_validate,y_validate), callbacks=[learning_rate_reduction])

In [ ]:
loss, accuracy = model_DO.evaluate(x_test, y_test, verbose=0)
predictions = model_DO.predict(x_test)
loss_v, accuracy_v = model_DO.evaluate(x_validate, y_validate, verbose=0)
loss_t, accuracy_t = model_DO.evaluate(x_train, y_train, verbose=0)
print("Training: accuracy = %f" % (accuracy_t))
print("Validation: accuracy = %f" % (accuracy_v))
print("Test: accuracy = %f" % (accuracy))

# ***VGG16 DB(3)***

In [ ]:
baseModel = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
headModel = baseModel.output

headModel_3 = DropBlock2D(block_size=3, keep_prob=0.5)(headModel)
headModel_3 = BatchNormalization()(headModel_3)
headModel_3 = AveragePooling2D()(headModel_3)
headModel_3 = Flatten(name="flatten")(headModel_3)
headModel_3 = Dense(64, activation="relu")(headModel_3)
headModel_3 = Dropout(0.5)(headModel_3)
headModel_3 = Dense(7, activation="softmax")(headModel_3)
model_3 = Model(inputs=baseModel.input, outputs=headModel_3)

for layer in baseModel.layers:
    layer.trainable = False
model_3.summary()

In [ ]:

model_3.compile(loss="categorical_crossentropy", optimizer=optimizer,
    metrics=["accuracy"])
H3 = model_3.fit(datagen.flow(x_train,y_train, batch_size=20),
                             epochs = 50, validation_data = (x_validate,y_validate), callbacks=[learning_rate_reduction])


In [ ]:
loss, accuracy = model_3.evaluate(x_test, y_test, verbose=0)
predictions = model_3.predict(x_test)
loss_v, accuracy_v = model_3.evaluate(x_validate, y_validate, verbose=0)
loss_t, accuracy_t = model_3.evaluate(x_train, y_train, verbose=0)
print("Training: accuracy = %f" % (accuracy_t))
print("Validation: accuracy = %f" % (accuracy_v))
print("Test: accuracy = %f" % (accuracy))

# ***VGG16 DB(5)***

In [ ]:

baseModel = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
headModel = baseModel.output
headModel_5 = DropBlock2D(block_size=5, keep_prob=0.5)(headModel)
headModel_5 = BatchNormalization()(headModel_5)
headModel_5 = AveragePooling2D()(headModel_5)
headModel_5 = Flatten(name="flatten")(headModel_5)
headModel_5 = Dense(64, activation="relu")(headModel_5)
headModel_5 = Dropout(0.5)(headModel_5)
headModel_5 = Dense(7, activation="softmax")(headModel_5)
model_5 = Model(inputs=baseModel.input, outputs=headModel_5)

model_5.summary()

In [ ]:
model_5.compile(loss="categorical_crossentropy", optimizer=optimizer,
    metrics=["accuracy"])
H5 = model_5.fit(datagen.flow(x_train,y_train, batch_size=20),
                             epochs = 50, validation_data = (x_validate,y_validate), callbacks=[learning_rate_reduction])


In [ ]:
loss, accuracy = model_5.evaluate(x_test, y_test, verbose=0)
predictions = model_5.predict(x_test)
loss_v, accuracy_v = model_5.evaluate(x_validate, y_validate, verbose=0)
loss_t, accuracy_t = model_5.evaluate(x_train, y_train, verbose=0)
print("Training: accuracy = %f" % (accuracy_t))
print("Validation: accuracy = %f" % (accuracy_v))
print("Test: accuracy = %f" % (accuracy))

# ***VGG16 DB(7)***

In [ ]:
baseModel = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
headModel = baseModel.output
headModel_7 = DropBlock2D(block_size=7, keep_prob=0.5)(headModel)
headModel_7 = BatchNormalization()(headModel_7)
headModel_7 = AveragePooling2D()(headModel_7)
headModel_7 = Flatten(name="flatten")(headModel_7)
headModel_7 = Dense(64, activation="relu")(headModel_7)
headModel_7 = Dropout(0.5)(headModel_7)
headModel_7 = Dense(7, activation="softmax")(headModel_7)
model_7 = Model(inputs=baseModel.input, outputs=headModel_7)
for layer in baseModel.layers:
    layer.trainable = False

model_7.summary()

In [ ]:
model_7.compile(loss="categorical_crossentropy", optimizer=optimizer,
    metrics=["accuracy"])
H7 = model_7.fit(datagen.flow(x_train,y_train, batch_size=20),
                             epochs = 50, validation_data = (x_validate,y_validate), callbacks=[learning_rate_reduction])

In [ ]:
loss, accuracy = model_7.evaluate(x_test, y_test, verbose=0)
predictions = model_7.predict(x_test)
loss_v, accuracy_v = model_7.evaluate(x_validate, y_validate, verbose=0)
loss_t, accuracy_t = model_7.evaluate(x_train, y_train, verbose=0)
print("Training: accuracy = %f" % (accuracy_t))
print("Validation: accuracy = %f" % (accuracy_v))
print("Test: accuracy = %f" % (accuracy))

# ***VGG16 SDO***

In [ ]:
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.layers import BatchNormalization
from keras_drop_block import DropBlock2D
baseModel = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
headModel = baseModel.output
headModel_SDO = SpatialDropout2D(0.5)(headModel)
headModel_SDO = BatchNormalization()(headModel_SDO)
headModel_SDO = AveragePooling2D()(headModel_SDO)
headModel_SDO = Flatten(name="flatten")(headModel_SDO)
headModel_SDO = Dense(64, activation="relu")(headModel_SDO)
headModel_SDO = Dropout(0.5)(headModel_SDO)
headModel_SDO = Dense(7, activation="softmax")(headModel_SDO)
model_SDO = Model(inputs=baseModel.input, outputs=headModel_SDO)

for layer in baseModel.layers:
    layer.trainable = False   

model_SDO.summary()

In [ ]:
model_SDO.compile(loss="categorical_crossentropy", optimizer=optimizer,
    metrics=["accuracy"])
HSDO = model_SDO.fit(datagen.flow(x_train,y_train, batch_size=20),
                             epochs = 50, validation_data = (x_validate,y_validate), callbacks=[learning_rate_reduction])

In [ ]:
loss, accuracy = model_SDO.evaluate(x_test, y_test, verbose=0)
predictions = model_SDO.predict(x_test)
loss_v, accuracy_v = model_SDO.evaluate(x_validate, y_validate, verbose=0)
loss_t, accuracy_t = model_SDO.evaluate(x_train, y_train, verbose=0)
print("Training: accuracy = %f" % (accuracy_t))
print("Validation: accuracy = %f" % (accuracy_v))
print("Test: accuracy = %f" % (accuracy))

# ***Plots***

In [ ]:
N = 50

plt.figure()
# plt.plot(np.arange(0, N), H.history["accuracy"], linewidth = 2, label="train_acc")
plt.plot(np.arange(0, N), H_DO.history["val_accuracy"], linewidth = 2, label="Drop Out")
plt.plot(np.arange(0, N), H3.history["val_accuracy"], linewidth = 2, label="Drop block 3")
plt.plot(np.arange(0, N), H5.history["val_accuracy"], linewidth = 2, label="Drop block 5")
plt.plot(np.arange(0, N), H7.history["val_accuracy"], linewidth = 2, label="Drop block 7")
plt.plot(np.arange(0, N), HSDO.history["val_accuracy"], linewidth = 2, label="Spatial Drop out")

plt.title("Training and Validation Accuracy ")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="upper left")